In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random
import numpy as np

from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device("cuda:0")

In [2]:
# MNIST dataset
cifar_train = datasets.CIFAR10(root='CIFAR10_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

cifar_test = datasets.CIFAR10(root='CIFAR10_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
learning_rate = 0.001
epochs = 15
batch_size = 128

valid_ratio = 0.1

In [4]:
num_train = len(cifar_train)
indices = list(range(num_train))
np.random.shuffle(indices)

split = int(np.floor(valid_ratio * num_train))

- SubsetRandomSampler(indices)
    - Samples elements randomly from a given list of indices, without replacement.

In [5]:
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [6]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=cifar_train,
                                          batch_size=batch_size,
                                          sampler=train_sampler,
                                          drop_last=True)
valid_loader = torch.utils.data.DataLoader(dataset=cifar_train,
                                          batch_size=batch_size,
                                          sampler=valid_sampler,
                                          drop_last=True)

In [7]:
def vaild_loss(model_valid):
    avg_loss = 0
    with torch.no_grad():
        model_valid.eval()
        for images, labels in valid_loader:
            pred = model_valid(images.to(device))
            criterion = nn.CrossEntropyLoss().to(device)
            loss = criterion(pred, labels.to(device))
            avg_loss+= loss.item()
    
    return avg_loss / len(valid_loader)

$$          H,W_{out} = \left\lfloor\frac{H,W_{in}  + 2 \times \text{padding} - \text{dilation}
                    \times (\text{kernel_size} - 1) - 1}{\text{stride}} + 1\right\rfloor$$

- 마지막에 fc 연결하기 전에 test tensor 넣고 out.shape 찍어보기. 유용함

In [8]:
class BasicCNN_test(nn.Module):
    
    def __init__(self):
        super(BasicCNN_test, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
    
    def forward(self, x):
        out = self.layer1(x)
        print(out.shape)
        out = self.layer2(out)
        print(out.shape)
        return out

In [9]:
model = BasicCNN_test()

In [10]:
xx = torch.Tensor(1,3,32,32) # torch는 [channel, H, W]
out = model(xx)  
# fc는 64x6x6를 input으로 받아야 겠군.

torch.Size([1, 32, 15, 15])
torch.Size([1, 64, 6, 6])


In [11]:
class BasicCNN(nn.Module):
    
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.fc = nn.Linear(128*6*6, 10, bias=True)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1) # out.shape[0] : batch_size
        out = self.fc(out)
        return out

In [12]:
model = BasicCNN().to(device)

In [13]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# train my model
total_batch = len(train_loader)
model.train()
best_valid = np.inf

for epoch in range(epochs):
    avg_loss = 0

    for X, Y in train_loader:
        X = X.to(device) #img
        Y = Y.to(device) #label

        pred = model(X)
        loss = criterion(pred, Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss / total_batch
    
    loss_valid = vaild_loss(model)
    print('Epoch:{}, train loss:{:.5f}, valid loss:{:.5f}'.format(epoch, avg_loss, loss_valid))

    if loss_valid < best_valid:
        torch.save(model.state_dict(), './saves/models/basicCNN.pth')
        best_valid = loss_valid

Epoch:0, train loss:1.59295, valid loss:1.42352
Epoch:1, train loss:1.26292, valid loss:1.26442
Epoch:2, train loss:1.12640, valid loss:1.12634
Epoch:3, train loss:1.04102, valid loss:1.05506
Epoch:4, train loss:0.98192, valid loss:1.02855
Epoch:5, train loss:0.93482, valid loss:0.99353
Epoch:6, train loss:0.89247, valid loss:1.00706
Epoch:7, train loss:0.85487, valid loss:0.96809
Epoch:8, train loss:0.83248, valid loss:0.95905
Epoch:9, train loss:0.80385, valid loss:0.93662
Epoch:10, train loss:0.77483, valid loss:0.94027
Epoch:11, train loss:0.75683, valid loss:0.92156
Epoch:12, train loss:0.73483, valid loss:0.93454
Epoch:13, train loss:0.71832, valid loss:0.92920
Epoch:14, train loss:0.69110, valid loss:0.96157


In [15]:
test_loader = torch.utils.data.DataLoader(dataset=cifar_test, batch_size=128, shuffle=False)

In [16]:
correct = 0
total = 0
model_test = BasicCNN().to(device)
model_test.load_state_dict(torch.load('./saves/models/basicCNN.pth'))

with torch.no_grad():
    model_test.eval()
    for images, labels in test_loader:
        output = model_test(images.to(device))
        _, predicted = torch.max(output, dim=1)        
        total += labels.size(0)
        correct += (predicted ==labels.to(device)).sum().item()

print('Accuracy on test images with BasicCNN:', (correct / total))

Accuracy on test images with BasicCNN: 0.6941


In [17]:
class CNN_Batchnorm(nn.Module):
    
    def __init__(self):
        super(CNN_Batchnorm, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.fc = nn.Sequential(
            nn.Linear(128*5*5, 128, bias=True),
            nn.BatchNorm1d(128),
            nn.Linear(128, 10, bias=True))
        
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0],-1)
        out = self.fc(out)
        return out

In [18]:
model = CNN_Batchnorm().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
total_batch = len(train_loader)
model.train()
best_valid = np.inf

for epoch in range(epochs):
    avg_loss = 0

    for X, Y in train_loader:
        X = X.to(device) #img
        Y = Y.to(device) #label
        
        pred = model(X)
        loss = criterion(pred, Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss / total_batch

    loss_valid = vaild_loss(model)
    print('Epoch:{}, train loss:{:.5f}, valid loss:{:.5f}'.format(epoch, avg_loss, loss_valid))

    if loss_valid < best_valid:
        torch.save(model.state_dict(), './saves/models/CNN_Batchnorm.pth')
        best_valid = loss_valid

Epoch:0, train loss:1.16244, valid loss:0.98175
Epoch:1, train loss:0.89215, valid loss:0.91366
Epoch:2, train loss:0.75818, valid loss:0.79455
Epoch:3, train loss:0.66787, valid loss:0.75104
Epoch:4, train loss:0.60049, valid loss:0.76877
Epoch:5, train loss:0.55054, valid loss:0.73790
Epoch:6, train loss:0.49906, valid loss:0.75485
Epoch:7, train loss:0.46275, valid loss:0.81312
Epoch:8, train loss:0.41925, valid loss:0.82009
Epoch:9, train loss:0.39304, valid loss:0.88460
Epoch:10, train loss:0.35816, valid loss:0.90321
Epoch:11, train loss:0.32785, valid loss:0.90632
Epoch:12, train loss:0.29952, valid loss:0.95983
Epoch:13, train loss:0.27413, valid loss:1.02390
Epoch:14, train loss:0.25732, valid loss:0.95564


In [20]:
correct = 0
total = 0
model_test = CNN_Batchnorm().to(device)
model_test.load_state_dict(torch.load('./saves/models/CNN_Batchnorm.pth'))

with torch.no_grad():
    model_test.eval()
    for images, labels in test_loader:
        output = model_test(images.to(device))
        _, predicted = torch.max(output, dim=1)        
        total += labels.size(0)
        correct += (predicted ==labels.to(device)).sum().item()

print('Accuracy on test images with Batchnorm CNN:', (correct / total))

Accuracy on test images with Batchnorm CNN: 0.7624
